In [30]:
human_messages = []
idx = 1
for item in context_json["content_text"]:
    index = "예시문항 " + str(idx) + "번"
    example = "<보기> " + item["example"]
    selectors = item["selector"]
    options = []
    for key, value in selectors.items():
        temp = f"{key}.{value}"
        options.append(temp)
    options_str = ", ".join(options)
    answer_key = next(iter(item["answer"]))
    answer_value = item["answer"][answer_key]
    answer = f"[정답 : {answer_key}.{answer_value}]"
    prompt = f"{index} : {example} (선택지: {options_str}) {answer}"
    human_messages.append(prompt)
    idx += 1
human_messages = "\n ".join(human_messages)
human_messages



'예시문항 1번 : <보기> 오늘은 월요일입니다. 내일은 화요일입니다. (선택지: 1.날짜, 2.휴일, 3.요일, 4.장소) [정답 : 3.요일]\n 예시문항 2번 : <보기> 사과가 맛있습니다. 바나나도 맛있습니다. (선택지: 1.날씨, 2.과일, 3.직업, 4.나이) [정답 : 2.과일]\n 예시문항 3번 : <보기> 여름에는 덥습니다. 겨울에는 추웁니다. (선택지: 1.계절, 2.날씨, 3.옷, 4.음식) [정답 : 1.계절]\n 예시문항 4번 : <보기> 가게에 갑니다. 모자를 삽니다. (선택지: 1.쇼핑, 2.공부, 3.이름, 4.음식) [정답 : 1.쇼핑]\n 예시문항 5번 : <보기> 7월에는 수업이 없습니다. 학교에 안 갑니다. (선택지: 1.계절, 2.방학, 3.여행, 4.위치) [정답 : 2.방학]'

In [ ]:
dict = {
      "example": "https://topikkorea.s3.amazonaws.com/topik_image/b805aea2-163f-11ef-ac46-e5a43cd400fd.png",
      "questions":[
        {
          "question" : "이 글을 쓴 의도와 가장 가까운 것을 고르십시오.",
          "selector": ["음악회를 계획하려고","음악회에 초대하려고","음악회 참석을 확인하려고","음악회 참석에 감사하려고"],
          "answer": "음악회에 초대하려고"
        },
        {
          "question" : "이 글의 내용과 같은 것을 고르십시오.",
          "selector": ["초콜릿을 먹으면 기분이 좋아집니다.","목이 마를 때 초콜릿을 먹는 것이 좋습니다.","가수들은 보통 공연 전에 초콜릿을 먹습니다.","말을 많이 해야 할 때 초콜릿을 먹으면 좋습니다."],
          "answer": "초콜릿을 먹으면 기분이 좋아집니다."
        }
      ]
    }

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
# Create the first prompt template
sys_prompt: ChatPromptTemplate = ChatPromptTemplate(
    input_variables=["original_sentence", "desired_language"],
    template="""You are a language translater, an English speaker wants to translate/
    {original_sentence} to {desired_language}. Tell him the corrent answer."""
)

system_message_prompt = SystemMessagePromptTemplate(prompt=sys_prompt)

generator_prompt: ChatPromptTemplate = ChatPromptTemplate(
    input_variables=["original_sentence", "desired_language"],
    template="Translate {original_sentence} to {desired_language}"
)
generator_message_prompt = HumanMessagePromptTemplate(prompt=generator_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, generator_message_prompt])
    
print(chat_prompt)
print("="*100)
print(chat_prompt.format(original_sentence="I love Pizza!", desired_language="French"))

In [16]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

response_schemas = [ResponseSchema(name='example', description="<보기>"),
ResponseSchema(name='answer_1', description="1.단어"),
ResponseSchema(name='answer_2', description="2.단어"),
ResponseSchema(name='answer_3', description="3.단어"),
ResponseSchema(name='answer_4', description="4.단어"),
ResponseSchema(name='solution', description="번호.단어"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"example": string  // <보기>
	"answer_1": string  // 1.단어
	"answer_2": string  // 2.단어
	"answer_3": string  // 3.단어
	"answer_4": string  // 4.단어
	"solution": string  // 번호.단어
}
```


In [12]:
from langchain.prompts import PromptTemplate
from langchain.prompts import load_prompt
from langchain.chains import LLMChain
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

response_schemas = [ResponseSchema(name='example', description="<보기>"),
ResponseSchema(name='answer_1', description="1.단어"),
ResponseSchema(name='answer_2', description="2.단어"),
ResponseSchema(name='answer_3', description="3.단어"),
ResponseSchema(name='answer_4', description="4.단어"),
ResponseSchema(name='solution', description="번호.단어"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

prompt = load_prompt("prompt_type_prompt_templates/reading_1_problem_type_1.json")
prompt = PromptTemplate(
    input_variables=["what"],
    template="{what} 이름 세 개를 추천해줘. \n{format_instructions}",
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)
chain = LLMChain(llm=chatgpt, prompt=prompt, output_parser=output_parser)
chain.predict(what="새로 만든 거대 언어 모델")


['Languatron', 'LexiMax', 'Vocabzilla']

In [77]:
from langchain.prompts import load_prompt
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

prompt = load_prompt("problem_type_prompt_templates/reading_1_problem_type_1.json")

system_message_prompt = SystemMessagePromptTemplate.from_template(
    "당신은 한국어 능력을 평가하는 문제의 출제자입니다. 예시문항을 참조하여 무엇에 대한 내용인지 서술하고 <보기>와 같이 알맞은 것을 고르는 문제를 출제해주세요. 보기의 내용에 포함된 단어와 정답의 단어가 동일하면 안된다. 주제는 주어진 예시 문항과 비슷해야 한다."
)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    "예시문항 : <보기> {example} 1.{answer_1} 2.{answer_2} 3.{answer_3} 4.{answer_4} | 답: {solution}"
)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)
print(chat_prompt.format_messages(example="오늘은 월요일입니다. 내일은 화요일입니다.", answer_1="날짜", answer_2="휴일", answer_3="요일", answer_4="장소", solution="3.요일"))
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
llm(chat_prompt.format_messages(example="오늘은 월요일입니다. 내일은 화요일입니다.", answer_1="날짜", answer_2="휴일", answer_3="요일", answer_4="장소", solution="3.요일", text="예시문항"))


[SystemMessage(content='당신은 한국어 능력을 평가하는 문제의 출제자입니다. 예시문항을 참조하여 무엇에 대한 내용인지 서술하고 <보기>와 같이 알맞은 것을 고르는 문제를 출제해주세요. 보기의 내용에 포함된 단어와 정답의 단어가 동일하면 안된다. 주제는 주어진 예시 문항과 비슷해야 한다.'), HumanMessage(content='예시문항 : <보기> 오늘은 월요일입니다. 내일은 화요일입니다. 1.날짜 2.휴일 3.요일 4.장소 | 답: 3.요일')]


AIMessage(content='문제 : <보기> "사과, 바나나, 수박은 어떤 과일인가요?" 1.과일 2.채소 3.육류 4.음료 | 답: 1.과일', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 204, 'total_tokens': 267}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-8360152b-93ed-4d66-95a5-ed7f983d6e31-0')

In [2]:
import pandas as pd
import csv

file = pd.read_csv('KoreanVocaDict.txt', delimiter = '\t')
file

,순위,단어,품사,풀이,등급
0,1195.0,가게,명,NaN,A
1,898.0,가격03,명,價格,B
2,2986.0,가구03,명,家口,C
3,7434.0,가구04,명,家具,B
4,4964.0,가까워지다,동,NaN,B
...,...,...,...,...,...
5960,489.0,힘들다,형,NaN,A
5961,10357.0,힘들어하다,동,NaN,C
5962,3305.0,힘쓰다,동,NaN,C
5963,9013.0,힘없이,부,NaN,C


In [3]:
file["단어"] = file["단어"].str.replace("\d+", "", regex=True)

,순위,단어,품사,풀이,등급
0,1195.0,가게,명,NaN,A
1,898.0,가격,명,價格,B
2,2986.0,가구,명,家口,C
3,7434.0,가구,명,家具,B
4,4964.0,가까워지다,동,NaN,B
...,...,...,...,...,...
5960,489.0,힘들다,형,NaN,A
5961,10357.0,힘들어하다,동,NaN,C
5962,3305.0,힘쓰다,동,NaN,C
5963,9013.0,힘없이,부,NaN,C


In [4]:
voca_list = file[file["품사"] == "명"]
voca_list

,순위,단어,품사,풀이,등급
0,1195.0,가게,명,NaN,A
1,898.0,가격,명,價格,B
2,2986.0,가구,명,家口,C
3,7434.0,가구,명,家具,B
6,3560.0,가까이,명,NaN,B
...,...,...,...,...,...
5950,2431.0,희곡,명,戱曲,C
5952,1529.0,희망,명,希望,B
5954,4963.0,희생,명,犧牲,C
5956,1999.0,흰색,명,-色,A


In [5]:
voca_unique = voca_list.drop_duplicates(subset=['단어'])
voca_unique

,순위,단어,품사,풀이,등급
0,1195.0,가게,명,NaN,A
1,898.0,가격,명,價格,B
2,2986.0,가구,명,家口,C
6,3560.0,가까이,명,NaN,B
10,5071.0,가난,명,NaN,C
...,...,...,...,...,...
5950,2431.0,희곡,명,戱曲,C
5952,1529.0,희망,명,希望,B
5954,4963.0,희생,명,犧牲,C
5956,1999.0,흰색,명,-色,A


In [6]:
df_sorted = voca_unique.sort_values(by='순위')

# '단어' 열만 추출
words = df_sorted['단어'].tolist()

# 단어 리스트를 텍스트 파일로 저장
with open('words_list.txt', 'w', encoding='utf-8') as f:
    for word in words:
        f.write(f"{word}\n")


In [250]:
data = {'example': '육아는 부모에게 큰 책임과 행복을 안겨줍니다. 아이들과 함께 보내는 시간은 소중하고 가치있는 경험이 될 수 있습니다. 육아는 아이들의 성장과 발달에 중요한 영향을 미치며, 가정의 행복과 안정을 ( ㉠ ). 부모의 사랑과 배려가 아이들에게는 큰 힘이 됩니다.',
 'questions': [{'question': '( ㉠ )에 들어갈 말로 가장 알맞은 것을 고르십시오.',
   'selector': ['방해합니다', '가져옵니다', '무시합니다', '방관합니다'],
   'answer': '가져옵니다',
   'eval_answer': '가져옵니다',
   'eval_explain': '육아는 부모에게 큰 책임과 행복을 안겨줍니다. 아이들과 함께 보내는 시간은 소중하고 가치있는 경험이 될 수 있습니다. 육아는 아이들의 성장과 발달에 중요한 영향을 미치며, 가정의 행복과 안정을 책임집니다. 부모의 사랑과 배려가 아이들에게는 큰 힘이 됩니다.'},
  {'question': '<보기>의 내용과 맞는 것을 고르십시오.',
   'selector': ['육아는 부모에게 큰 책임과 행복을 안겨줍니다.',
    '아이들과 함께 보내는 시간은 소중하지 않습니다.',
    '육아는 아이들의 성장과 발달에 영향을 미치지 않습니다.',
    '부모의 사랑과 배려는 아이들에게 큰 힘이 되지 않습니다.'],
   'answer': '육아는 부모에게 큰 책임과 행복을 안겨줍니다.',
   'eval_answer': '육아는 부모에게 큰 책임과 행복을 안겨줍니다.',
   'eval_explain': '이 문장은 육아의 긍정적인 측면을 강조하고 있으며, 문맥에 맞는 올바른 서술입니다. 나머지 선택지는 부정적인 의미를 담고 있어 문맥에 맞지 않습니다.'}]}
import json
json_val = json.dumps(data, ensure_ascii=False)
json_data= json.loads(json_val)
json_data

{'example': '육아는 부모에게 큰 책임과 행복을 안겨줍니다. 아이들과 함께 보내는 시간은 소중하고 가치있는 경험이 될 수 있습니다. 육아는 아이들의 성장과 발달에 중요한 영향을 미치며, 가정의 행복과 안정을 ( ㉠ ). 부모의 사랑과 배려가 아이들에게는 큰 힘이 됩니다.',
 'questions': [{'question': '( ㉠ )에 들어갈 말로 가장 알맞은 것을 고르십시오.',
   'selector': ['방해합니다', '가져옵니다', '무시합니다', '방관합니다'],
   'answer': '가져옵니다',
   'eval_answer': '가져옵니다',
   'eval_explain': '육아는 부모에게 큰 책임과 행복을 안겨줍니다. 아이들과 함께 보내는 시간은 소중하고 가치있는 경험이 될 수 있습니다. 육아는 아이들의 성장과 발달에 중요한 영향을 미치며, 가정의 행복과 안정을 책임집니다. 부모의 사랑과 배려가 아이들에게는 큰 힘이 됩니다.'},
  {'question': '<보기>의 내용과 맞는 것을 고르십시오.',
   'selector': ['육아는 부모에게 큰 책임과 행복을 안겨줍니다.',
    '아이들과 함께 보내는 시간은 소중하지 않습니다.',
    '육아는 아이들의 성장과 발달에 영향을 미치지 않습니다.',
    '부모의 사랑과 배려는 아이들에게 큰 힘이 되지 않습니다.'],
   'answer': '육아는 부모에게 큰 책임과 행복을 안겨줍니다.',
   'eval_answer': '육아는 부모에게 큰 책임과 행복을 안겨줍니다.',
   'eval_explain': '이 문장은 육아의 긍정적인 측면을 강조하고 있으며, 문맥에 맞는 올바른 서술입니다. 나머지 선택지는 부정적인 의미를 담고 있어 문맥에 맞지 않습니다.'}]}

In [28]:
from util import find_random_question

exam_type = 'TOPIK_1_READING'
problem_type = 'READING_1_PROBLEM_TYPE_1'
result = find_random_question(exam_type, problem_type, detail_type=2, count=1)
for r in result:
    r['example'] = [r['example']]
result.pop()

{'example': ['오늘은 제 생일입니다.친구들을 ( ).'],
 'selector': ['좋아했습니다', '초대했습니다', '가르쳤습니다', '구경했습니다'],
 'answer': '초대했습니다'}

In [1]:
import re

def convert_string_to_list_format(input_string):
    # 정규 표현식을 사용하여 (가), (나), (다), (라) 등을 기준으로 분리합니다.
    sentences = re.split(r'\s*(\([가-힣]\))\s*', input_string)
    
    # 빈 문자열과 None을 제거하고 유효한 문장과 부호를 페어로 묶습니다.
    sentences = [s for s in sentences if s and s != ' ']
    paired_sentences = [(sentences[i], sentences[i+1]) for i in range(0, len(sentences), 2)]
    
    # 지정된 형식으로 문자열을 작성합니다.
    result = "[["
    for pair in paired_sentences:
        result += f"'{pair[0]} {pair[1]}', "
    
    # 마지막 문장의 끝에 있는 ', '를 제거하고 닫는 대괄호를 추가합니다.
    result = result.rstrip(', ') + "]]"
    
    return result

# 예제 입력
input_string = "(가) 그런 다음, 수집한 정보를 바탕으로 독자가 이해하기 쉽게 구성해야 한다. (나) 기사 작성은 정보 전달을 위한 중요한 작업이다. (다) 기자는 사건이나 이슈에 대한 연구와 인터뷰를 통해 사실을 수집한다. (라) 마지막으로, 기사는 독자의 관심을 끌고 신뢰를 쌓기 위해 객관적이고 정확해야 한다."

# 함수 호출
output_string = convert_string_to_list_format(input_string)
print(output_string)

[['(가) 그런 다음, 수집한 정보를 바탕으로 독자가 이해하기 쉽게 구성해야 한다.', '(나) 기사 작성은 정보 전달을 위한 중요한 작업이다.', '(다) 기자는 사건이나 이슈에 대한 연구와 인터뷰를 통해 사실을 수집한다.', '(라) 마지막으로, 기사는 독자의 관심을 끌고 신뢰를 쌓기 위해 객관적이고 정확해야 한다.']]


In [173]:
import os

def clean_text(text):
    if text.startswith('http'):
        return text
    pattern = r':\s*(.*)$'
    match = re.search(pattern, text)
    if match:
        return match.group(1).strip()
    text = re.sub(r'\d+\.', '', text)  # 숫자 다음에 오는 점 (예: "1.", "2.") 제거
    text = re.sub(r'\n', ' ', text)  # 개행 문자를 공백으로 대체
    return text.strip()

def assign_scores(total_score, num_questions):
    if total_score == 4 and num_questions == 2:
        return [2, 2]
    if total_score == 5 and num_questions == 2:
        return [2, 3]
    if total_score == 6 and num_questions == 2:
        return [3, 3]
    
    scores = []
    remaining_score = total_score
    
    for i in range(num_questions):
        if remaining_score - 3 >= (num_questions - len(scores) - 1) * 2:
            scores.append(3)
            remaining_score -= 3
        else:
            scores.append(2)
            remaining_score -= 2
    #print(scores)
    return scores

def exam_scoring(json_data):
    scores_per_type = [6, 14, 9, 8, 8, 19, 5, 5, 26]  # 각 유형별 총 점수

    problem_num = 0
    for type_index, questions in enumerate(json_data):
        total_score = scores_per_type[type_index]
        current_score = total_score
        num_questions = len(questions)
        if type_index in [5, 7, 8]:  # 유형 번호가 6, 8, 9인 경우 (인덱스는 0부터 시작하므로 5, 7, 8 사용)
            for question in questions:
                question["example"] = clean_text(question["example"])
                question["score"] = 5 
                #print(f"Type {type_index + 1} problem {problem_num + 1} score: {current_score}")
                sub_total_score = question["score"]  # 하위 문제에 할당된 총 점수
                num_sub_questions = len(question["questions"])  # 하위 문제의 수
                sub_scores = assign_scores(5, num_sub_questions)  # 하위 문제에 점수 할당
                for sub_question_index, sub_question in enumerate(question["questions"]):
                    sub_question["score"] = sub_scores[sub_question_index]  # 하위 문제에 점수 할당
                    #exam_score += sub_question["score"]
                    current_score -= sub_question["score"]
                    for i in range(len(sub_question["selector"])):
                        sub_question["selector"][i] = clean_text(sub_question["selector"][i])
                    sub_question["answer"] = clean_text(sub_question["answer"])
                    sub_question["eval_answer"] = clean_text(sub_question["eval_answer"])
                    
        else:
            scores = assign_scores(total_score, num_questions)
            for question_index, question in enumerate(questions):
                question["example"] = clean_text(question["example"])
                for i in range(len(question["selector"])):
                        question["selector"][i] = clean_text(question["selector"][i])
                question["answer"] = clean_text(question["answer"])
                question["eval_answer"] = clean_text(question["eval_answer"])
                question["score"] = scores[question_index]
                #exam_score+=question["score"]
                problem_num += 1
        #print(f"Type {type_index + 1} score: {total_score}")
    return json_data

def example_preprocess(json_data):
    temp = []
    for item in json_data[10:12]:
        for r in item:
            temp.append(r)
    del json_data[10:12]
    json_data.insert(10, temp) 

    temp = []
    for item in json_data[1:4]:
        for r in item:
            temp.append(r)

    del json_data[1:4]
    json_data.insert(1, temp)
    json_data = exam_scoring(json_data)
    return json_data


from datetime import datetime
import glob

folder_path = '/home/shinbg/topik-korea-data/data/problem_data' 

# 'generate_exam_'로 시작하고 '.json'으로 끝나는 모든 파일을 검색합니다.
file_pattern = os.path.join(folder_path, "generate_exam_*.json")
files = glob.glob(file_pattern)

if not files:
    print("JSON 파일을 찾을 수 없습니다.")
else:
    Exam_cost_list = []
    for file in files:
        with open(file, "r") as json_file:
            exam_cost = []
            json_data = json.load(json_file)
            json_data = example_preprocess(json_data)
            for item in json_data:
                for question in item:
                    exam_cost.append(question['total_cost'])
            print("Exam Total Cost: ", sum(exam_cost))
            print("Problem Avg Cost: ", sum(exam_cost)/len(exam_cost))
            Exam_cost_list.append(sum(exam_cost))
    print("Total TESTSET Cost: ", sum(Exam_cost_list))
    print("Total TESTSET Avg Cost: ", sum(Exam_cost_list)/len(Exam_cost_list))

Exam Total Cost:  396.23849999999993
Problem Avg Cost:  13.207949999999999
Exam Total Cost:  381.4829999999999
Problem Avg Cost:  12.716099999999996
Exam Total Cost:  387.9359999999999
Problem Avg Cost:  12.931199999999997
Exam Total Cost:  411.07499999999993
Problem Avg Cost:  13.702499999999997
Exam Total Cost:  400.14000000000004
Problem Avg Cost:  13.338000000000001
Exam Total Cost:  358.50600000000003
Problem Avg Cost:  11.9502
Exam Total Cost:  420.2819999999999
Problem Avg Cost:  14.009399999999998
Exam Total Cost:  392.067
Problem Avg Cost:  13.068900000000001
Exam Total Cost:  374.0445000000001
Problem Avg Cost:  12.468150000000003
Exam Total Cost:  386.451
Problem Avg Cost:  12.8817
Total TESTSET Cost:  3908.223
Total TESTSET Avg Cost:  390.8223


In [40]:
import os
from datetime import datetime
import glob
import json
from Problem_Type_Template.reading_1_problem_type_7 import generate_reading_1_problem_type_7

folder_path = '/home/shinbg/topik-korea-data/data/problem_data' 

# 'generate_exam_'로 시작하고 '.json'으로 끝나는 모든 파일을 검색합니다.
file_pattern = os.path.join(folder_path, "generate_exam_*.json")
files = glob.glob(file_pattern)

if not files:
    print("JSON 파일을 찾을 수 없습니다.")
else:
    for file in files[:1]:
        with open(file, "r") as json_file:
            json_data = json.load(json_file)
            for i in range(len(json_data)):
                    # json_data[i] = [generate_reading_1_problem_type_7()]
                print(json_data[i])
            # with open(file, 'w') as outfile:
            #     json.dump(json_data, outfile, indent=4, ensure_ascii=False)

[{'example': '이 제품은 훌륭합니다. 서비스가 매우 만족스럽습니다.', 'selector': ['분석', '검토', '진단', '평가'], 'answer': '평가', 'eval_answer': '평가', 'eval_explain': "문장에서 제품과 서비스에 대한 긍정적인 의견을 표현하고 있으므로, '평가'가 가장 적합한 단어입니다. '분석', '검토', '진단'은 주로 문제를 찾거나 개선점을 찾기 위한 과정에서 사용됩니다.", 'total_cost': 12.204}, {'example': '근로자는 열심히 일합니다.   근로자는 주말에 쉽니다.', 'selector': ['사회', '생활', '경제', '문화'], 'answer': '생활', 'eval_answer': '생활  ', 'eval_explain': "근로자의 일과 휴식은 일상생활과 관련된 내용이므로 '생활'이 가장 적합한 단어입니다.", 'total_cost': 11.61}, {'example': '신선한 해산물 맛있다. 해산물 요리 추천합니다.', 'selector': ['도구', '동물', '음식', '의류'], 'answer': '음식', 'eval_answer': '음식', 'eval_explain': "문장에서 '신선한 해산물'과 '해산물 요리'가 언급되고 있으므로, 이는 '음식'과 관련이 있습니다.", 'total_cost': 11.853}]
[{'example': '월요일은 바쁩니다. (   )은 좋습니다.', 'selector': ['주말', '토요일', '일요일', '금요일'], 'answer': '금요일', 'eval_answer': '금요일', 'eval_explain': '월요일은 바쁩니다. 금요일은 좋습니다.', 'total_cost': 6.264}, {'example': '밥이 잘 지어졌어요.   (   )를 깨끗이 씻어요.', 'selector': ['푸성귀', '야채', '나물', '채소'], 'answer': '야채', 'eval_answer': '야채

In [2]:
import glob
import os
import json
from datetime import datetime
import requests
import time
folder_path = '/home/shinbg/topik-korea-data/data/reading_2/exam_data' 
url = "http://waterboom.iptime.org:8080/exam"
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ASDLFNAFO!#!ENADSLFSADFSANVSAL!ADSLKFJSNAVLSANVSNNVASDJVASLJSAFLQNA>S:SA:WNCW:ASAFAWE'}

# 'generate_exam_'로 시작하고 '.json'으로 끝나는 모든 파일을 검색합니다.
file_pattern = os.path.join(folder_path, "generate_exam_*.json")
files = glob.glob(file_pattern)

# print(files)
if not files:
    print("JSON 파일을 찾을 수 없습니다.")
else:
    def extract_datetime_from_filename(filename):
        base_name = os.path.basename(filename)
        datetime_str = '_'.join(base_name.split('_')[-2:]).split('.')[0]
        return datetime.strptime(datetime_str, '%Y%m%d_%H%M')

    files = sorted(files, key=extract_datetime_from_filename)
    i = 1
    for file in files:
        with open(file, "r", encoding='utf-8-sig') as json_file:
            json_data = json.load(json_file)
            json_data['title'] = json_data['title'].replace("N", str(i),1)
            print(json_data['title'])
            response = requests.post(url, data=json.dumps(json_data), headers=headers)
            print(response)
            if response.status_code == 201:
                print('Success')
            else:
                print('Failed')
                raise ValueError('Failed')
            i+=1
            time.sleep(1)
                        

Generated_TEST1_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST2_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST3_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST4_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST5_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST6_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST7_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST8_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST9_TOPIK_2_READING
<Response [201]>
Success
Generated_TEST10_TOPIK_2_READING
<Response [201]>
Success
